In [111]:
import torch
import numpy as np
import torchvision
import pandas as pd
from PIL import Image
import torch.nn as nn
from pathlib import Path
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [112]:
#read the csv
#column Breed shows the y with the image name
data_csv = pd.read_csv('dataset_CV.csv')
data_csv

,Unnamed: 0,DogId,Breed,Image,ImageNumber
0,9242,9473,AMERICAN STAFFORDSHIRE BULL TERRIER,89f3c062-8928-487b-b7c0-8ab50f44641e.jpg,1
1,9243,9473,AMERICAN STAFFORDSHIRE BULL TERRIER,e9e33b00-5ec4-4742-ac02-b9ee2a88314f.jpg,2
2,9244,9473,AMERICAN STAFFORDSHIRE BULL TERRIER,2a81e50c-efdb-4e98-9220-d6945753731d.jpg,3
3,9245,9473,AMERICAN STAFFORDSHIRE BULL TERRIER,0609e883-2c18-40b2-a4a5-8a90754a94f6.jpg,4
4,9507,9548,AMERICAN STAFFORDSHIRE BULL TERRIER,10fb3092-842a-466b-a3db-3c88f624c6f4.jpg,1
...,...,...,...,...,...
4145,117179,38513,BORDER COLLIE,e3957a7c-9289-4441-bdbd-d2289bef745d.jpg,5
4146,117180,38513,BORDER COLLIE,48afd0ab-baf2-4ee8-9584-9563cec529e7.jpg,6
4147,117181,38513,BORDER COLLIE,8f8c4ea0-7426-4188-bb6c-94fa1ecb1e33.jpg,7
4148,117182,38513,BORDER COLLIE,cce5499f-eb04-4d8c-8d3f-c9c9bee16c5c.jpg,8


In [113]:
# preprocessing
trans = transforms.Compose([
    transforms.Resize((80, 80)),
    transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),#values anpassen
    #transforms.AugMix(),
    ])

location = r"dog_breed_photos"
def create_data(path):
    image = Image.open(Path(location) / path)
    #img = image.int()
    img = trans(image)
    image.close()
    return img
    
data_csv['data'] = data_csv['Image'].apply(lambda image: create_data(image))
data_csv

,Unnamed: 0,DogId,Breed,Image,ImageNumber,data
0,9242,9473,AMERICAN STAFFORDSHIRE BULL TERRIER,89f3c062-8928-487b-b7c0-8ab50f44641e.jpg,1,"[[[tensor(0.4863), tensor(0.5255), tensor(0.57..."
1,9243,9473,AMERICAN STAFFORDSHIRE BULL TERRIER,e9e33b00-5ec4-4742-ac02-b9ee2a88314f.jpg,2,"[[[tensor(1.), tensor(1.), tensor(1.), tensor(..."
2,9244,9473,AMERICAN STAFFORDSHIRE BULL TERRIER,2a81e50c-efdb-4e98-9220-d6945753731d.jpg,3,"[[[tensor(1.), tensor(1.), tensor(1.), tensor(..."
3,9245,9473,AMERICAN STAFFORDSHIRE BULL TERRIER,0609e883-2c18-40b2-a4a5-8a90754a94f6.jpg,4,"[[[tensor(1.), tensor(1.), tensor(1.), tensor(..."
4,9507,9548,AMERICAN STAFFORDSHIRE BULL TERRIER,10fb3092-842a-466b-a3db-3c88f624c6f4.jpg,1,"[[[tensor(0.6863), tensor(0.6824), tensor(0.68..."
...,...,...,...,...,...,...
4145,117179,38513,BORDER COLLIE,e3957a7c-9289-4441-bdbd-d2289bef745d.jpg,5,"[[[tensor(0.1255), tensor(0.1333), tensor(0.18..."
4146,117180,38513,BORDER COLLIE,48afd0ab-baf2-4ee8-9584-9563cec529e7.jpg,6,"[[[tensor(0.0431), tensor(0.0745), tensor(0.12..."
4147,117181,38513,BORDER COLLIE,8f8c4ea0-7426-4188-bb6c-94fa1ecb1e33.jpg,7,"[[[tensor(0.2627), tensor(0.2667), tensor(0.30..."
4148,117182,38513,BORDER COLLIE,cce5499f-eb04-4d8c-8d3f-c9c9bee16c5c.jpg,8,"[[[tensor(0.7765), tensor(0.8235), tensor(0.63..."


In [114]:
#creat dataframe for loader
df = pd.concat([data_csv['data'], data_csv['Breed']], ignore_index=True, axis = 1)
df

,0,1
0,"[[[tensor(0.4863), tensor(0.5255), tensor(0.57...",AMERICAN STAFFORDSHIRE BULL TERRIER
1,"[[[tensor(1.), tensor(1.), tensor(1.), tensor(...",AMERICAN STAFFORDSHIRE BULL TERRIER
2,"[[[tensor(1.), tensor(1.), tensor(1.), tensor(...",AMERICAN STAFFORDSHIRE BULL TERRIER
3,"[[[tensor(1.), tensor(1.), tensor(1.), tensor(...",AMERICAN STAFFORDSHIRE BULL TERRIER
4,"[[[tensor(0.6863), tensor(0.6824), tensor(0.68...",AMERICAN STAFFORDSHIRE BULL TERRIER
...,...,...
4145,"[[[tensor(0.1255), tensor(0.1333), tensor(0.18...",BORDER COLLIE
4146,"[[[tensor(0.0431), tensor(0.0745), tensor(0.12...",BORDER COLLIE
4147,"[[[tensor(0.2627), tensor(0.2667), tensor(0.30...",BORDER COLLIE
4148,"[[[tensor(0.7765), tensor(0.8235), tensor(0.63...",BORDER COLLIE


In [115]:
# train, test split
train, test = train_test_split(df, test_size = 0.25, random_state=42,stratify=Y, shuffle = True)
train.reset_index(drop=True)

,0,1
0,"[[[tensor(0.4196), tensor(0.3882), tensor(0.36...",AMERICAN STAFFORDSHIRE BULL TERRIER
1,"[[[tensor(0.6824), tensor(0.7059), tensor(0.70...",BULL ARAB
2,"[[[tensor(0.7922), tensor(0.8235), tensor(0.86...",AMERICAN STAFFORDSHIRE BULL TERRIER
3,"[[[tensor(0.7255), tensor(0.6824), tensor(0.64...",GERMAN SHEPHERD
4,"[[[tensor(0.8549), tensor(0.8510), tensor(0.84...",GERMAN SHEPHERD
...,...,...
3107,"[[[tensor(0.4235), tensor(0.4510), tensor(0.45...",AMERICAN STAFFORDSHIRE BULL TERRIER
3108,"[[[tensor(0.6431), tensor(0.6314), tensor(0.59...",AMERICAN STAFFORDSHIRE BULL TERRIER
3109,"[[[tensor(0.7569), tensor(0.7686), tensor(0.77...",BULL ARAB
3110,"[[[tensor(0.4392), tensor(0.4431), tensor(0.43...",SIBERIAN HUSKY


In [116]:
#dataloader
train_loader = torch.utils.data.DataLoader(train, batch_size= 4, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size= 4, shuffle=True)

In [117]:
#Define model

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [118]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [119]:
train_loader

In [121]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data
    
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        print(f"current epoch: {epoch + 1}")


print('Finished Training')

KeyError: 1119

In [ ]:
#eval

In [ ]:
#visualize

In [ ]:
#plot muss noch angepasst werden
plt.rcParams["savefig.bbox"] = 'tight'
orig_img = df[42]
# if you change the seed, make sure that the randomly-applied transforms
# properly show that the image can be both transformed and *not* transformed!
torch.manual_seed(0)


def plot(imgs, with_orig=True, row_title=None, **imshow_kwargs):
    if not isinstance(imgs[0], list):
        # Make a 2d grid even if there's just 1 row
        imgs = [imgs]

    num_rows = len(imgs)
    num_cols = len(imgs[0]) + with_orig
    fig, axs = plt.subplots(nrows=num_rows, ncols=num_cols, squeeze=False)
    for row_idx, row in enumerate(imgs):
        row = [orig_img] + row if with_orig else row
        for col_idx, img in enumerate(row):
            ax = axs[row_idx, col_idx]
            ax.imshow(np.asarray(img), **imshow_kwargs)
            ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

    if with_orig:
        axs[0, 0].set(title='Original image')
        axs[0, 0].title.set_size(8)
    if row_title is not None:
        for row_idx in range(num_rows):
            axs[row_idx, 0].set(ylabel=row_title[row_idx])

    plt.tight_layout()